In [0]:
%pip install xgboost

In [0]:
%restart_python

In [0]:
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [0]:
# Load Dataset
df_spark = spark.table("olist.gold.gld_ml_features")

In [0]:
# Handle Null Values
df_spark = df_spark.fillna({
    "avg_review_score": 3,
    "avg_delivery_days": 7,
    "avg_delivery_delay": 0
})

In [0]:
# Convert Spark to Pandas
df = df_spark.toPandas()

In [0]:
# Required Columns
feature_cols = [
    "recency_days",
    "frequency",
    "avg_review_score",
    "avg_delivery_days",
    "avg_delivery_delay"
]

label_col = "high_value_customer"

In [0]:
# Train/Test Split
X = df[feature_cols]
y = df[label_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [0]:
with mlflow.start_run(run_name="Logistic_Regression"):

    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    y_pred_proba = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred_proba)

    mlflow.log_param("model", "logistic_regression")
    mlflow.log_metric("roc_auc", auc)

    mlflow.sklearn.log_model(model, "model")


2026/02/01 03:20:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [0]:
with mlflow.start_run(run_name="Random_Forest"):

    model = RandomForestClassifier(
        n_estimators=200,
        max_depth=10,
        random_state=42
    )

    model.fit(X_train, y_train)

    y_pred_proba = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred_proba)

    mlflow.log_param("model", "random_forest")
    mlflow.log_metric("roc_auc", auc)

    mlflow.sklearn.log_model(model, "model")


2026/02/01 03:20:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [0]:
with mlflow.start_run(run_name="Gradient_Boosting"):

    model = GradientBoostingClassifier(random_state=42)
    model.fit(X_train, y_train)

    y_pred_proba = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred_proba)

    mlflow.log_param("model", "gradient_boosting")
    mlflow.log_metric("roc_auc", auc)

    mlflow.sklearn.log_model(model, "model")


2026/02/01 03:20:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [0]:
with mlflow.start_run(run_name="XGBoost"):

    model = xgb.XGBClassifier(
        n_estimators=300,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="binary:logistic",
        eval_metric="auc",
        random_state=42
    )

    model.fit(X_train, y_train)

    y_pred_proba = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred_proba)

    mlflow.log_param("model", "xgboost")
    mlflow.log_param("n_estimators", 300)
    mlflow.log_param("max_depth", 6)
    mlflow.log_param("learning_rate", 0.05)

    mlflow.log_metric("roc_auc", auc)

    mlflow.xgboost.log_model(model, artifact_path="model")


/local_disk0/.ephemeral_nfs/envs/pythonEnv-2ad7ebb0-0c4f-45b8-8a04-6d5371fbec51/lib/python3.12/site-packages/xgboost/sklearn.py:1118: UserWarning: [03:20:54] WARNING: /workspace/src/c_api/c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  self.get_booster().save_model(fname)
2026/02/01 03:20:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
